# Spatial Statistics and Geospatial data visualization

## Key concepts in Geospatial data

### Raster and vector data

Raster data is a grid of pixels, each pixel representing a value. Raster data is often used to represent continuous data, such as elevation, temperature, or precipitation. Raster data is often stored as a file with a .tif extension.

Vector data is a collection of points, lines, and polygons. Vector data is often used to represent discrete data, such as the location of a city, the boundaries of a country, or the location of a river. Vector data is often stored as a file with a .shp extension.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rs
import rasterio.plot

import tempfile

import numpy as np
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages

rpy2.robjects.numpy2ri.activate()

In [ ]:
# Then, using the language of your choice (R or Python), design a non-interactive plot of French school districts

# Read in the data in .shp format
frenchAcademyDf = gpd.read_file('academies-20160209-shp/academies-20160209.shp')
frenchAcademyDf

In [ ]:
# Keep only metropolitan France
# Plot and color the data
colors = [plt.get_cmap('viridis')(i * 20) for i in range(len(frenchAcademyDf["wikipedia"]))]
frenchAcademyDf.clip([-5, 41, 10, 51]).plot(color=colors, edgecolor='black')

In [ ]:
# Find a shapefile representing the entire world, 
# and design a set of maps based on different pro-jections / coordinate reference systems (CRS) (ex.: Mercator, etc.).

# Read in the data in .shp format
worldDf = gpd.read_file('world-administrative-boundaries/world-administrative-boundaries.shp')
worldDf.plot()

In [ ]:
# Mercator projection
worldDf.to_crs(epsg=3395).plot(cmap='viridis')

In [ ]:
# Lambert Azimuthal Equal Area projection
worldDf.to_crs(epsg=9822).plot(cmap='viridis')

In [ ]:
# Polar Stereographic projection
worldDf.to_crs(epsg=3031).plot(cmap='viridis')

Write down a paragraph (in your notebook only) explaining the concept of CRS.

CRS means Coordinate Reference System and is a framework used to precisely measure locations on the surface of the Earth as coordinates. It is thus the application of the abstract mathematics of coordinate systems and analytic geometry to geographic space.

In [ ]:
# Download a raster dataset from the SRTM website (elevation data from the entire world, +-30 meters in precision). Design a 2D map of the area you selected. The size of the area of interest mostly depends upon your laptop’s computing power, since rendering a map can be CPU intensive.

# Read in the data in .tif format
img = rs.open('output_SRTMGL1.tif')

# Plot the data
rs.plot.show(img, cmap='terrain')

### Digital Elevation Models



In [ ]:
def rayshade(z, img_path=None, zscale=10, fov=0, theta=135, zoom=0.75, phi=45, windowsize=(1000, 1000)):
    
    # Output path.
    if not img_path:
        img_path = tempfile.NamedTemporaryFile(suffix='.png').name
    
    # Import needed packages.
    rayshader = rpackages.importr('rayshader')
    
    # Convert array to matrix.
    z = np.asarray(z)
    rows, cols, _ = z.shape
    z_mat = ro.r.matrix(z, nrow=rows, ncol=cols)
    ro.globalenv['elmat'] = z_mat
    
    # Save python state to r.
    ro.globalenv['img_path'] = img_path
    ro.globalenv['zscale'] = zscale
    ro.globalenv['fov'] = fov
    ro.globalenv['theta'] = theta
    ro.globalenv['zoom'] = zoom
    ro.globalenv['phi'] = phi
    ro.globalenv['windowsize'] = ro.IntVector(windowsize)
    
    # Do the render.
    ro.r('''
        elmat %>%
          sphere_shade(texture = "desert") %>%
          add_water(detect_water(elmat), color = "desert") %>%
          add_shadow(ray_shade(elmat, zscale = 3), 0.5) %>%
          add_shadow(ambient_shade(elmat), 0) %>%
          plot_3d(elmat, zscale = zscale, fov = fov, theta = theta, zoom = zoom, phi = phi, windowsize = windowsize)
        Sys.sleep(0.2)
        render_snapshot(img_path)
    ''')
    
    # Return path.
    return img_path

In [ ]:
SRTMG = plt.imread('output_SRTMGL1.tif')
img_path = rayshade(SRTMG)

from IPython.display import Image
Image(filename=img_path) 